In [ ]:
# Copyright 2024 Forusone(shins777@gmail.com)
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Function calling

* [Gemini fuction calling](https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/function-calling)
  * When submitting a prompt to the LLM, you also provide the model with a set of tools that it can use to respond to the user's prompt. For example, you could provide a function get_weather that takes a location parameter and returns information about the weather conditions at that location.
  * Call flow of gemini function calling
    * <image src='https://cloud.google.com/static/vertex-ai/generative-ai/docs/multimodal/images/function-calling.png' width=600>

## Set configuration

## Install  packages

In [1]:
%pip install --upgrade --quiet --user google-cloud-aiplatform

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 52.1 MB/s eta 0:00:00
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


## Authentication to access to GCP
* Only for Colab in Google Drive
* No need to do this process if in Colab Enteprise on Vertex AI.

In [2]:

# To use markdown for output data from LLM
from IPython.display import display, Markdown

# Use OAuth to access the GCP environment.
import sys
if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user()

## Lab execution


### Define constants

In [3]:
PROJECT_ID = "ai-hangsik"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}
MODEL_NAME = "gemini-1.5-flash-002" # @param {type:"string"}

### Import libraries

In [10]:
import vertexai

from vertexai.generative_models import (
    GenerationConfig,
    # GenerativeModel, replace by prview version.
    HarmBlockThreshold,
    HarmCategory,
    GenerationResponse,
    Tool,
    Part,
    ChatSession,
    FunctionDeclaration
)

from vertexai.preview.generative_models import (
    grounding,
    GenerativeModel
)
from vertexai.preview import (
    caching
)


### Initialize Vertex AI

In [5]:
# https://cloud.google.com/python/docs/reference/aiplatform/latest#initialization
vertexai.init(project=PROJECT_ID, location=LOCATION)

# https://cloud.google.com/vertex-ai/generative-ai/docs/reference/python/latest/vertexai.generative_models.GenerativeModel
model = GenerativeModel(MODEL_NAME)

### Helper functions - Function Tools

In [23]:
# Get capital city
def get_capital(
    country:str, ):

  """Retrieves the capital city name of the given country"""

  model = GenerativeModel(MODEL_NAME)
  prompt = f"Answer the capital city of {country}"

  responses = model.generate_content(
        prompt,
  )

  return responses.text

# Get exchange rate
def get_exchange_rate(
    currency_from: str = "USD",
    currency_to: str = "KRW",
    currency_date: str = "latest",
):
    """Retrieves the exchange rate between two currencies on a specified date."""
    import requests

    response = requests.get(
        f"https://api.frankfurter.app/{currency_date}?base=USD",
        # params={"from": currency_from, "to": currency_to},
    )
    return response.json()

### Define tools using functions

In [24]:
tool_get_capital = FunctionDeclaration(
    name="get_capital",
    description="Retrieves the capital city name of the given country",
    parameters={
        "type": "object",
        "properties": {
            "country": {"type": "string", "description": "name of country"}
        },
    },
)

tool_get_exchange_rate = FunctionDeclaration(
    name="get_exchange_rate",
    description="Retrieves the exchange rate between two currencies on a specified date.",
    parameters={
        "type": "object",
        "properties": {
            "currency_from": {"type": "string", "description": "Source currency"},
            "currency_to": {"type": "string", "description": "Target currency"},
            "currency_date": {"type": "string", "description": "Current date"}
        },
    },
)

### Register tools to Model

In [25]:
# Register Tools
search_tool = Tool(
    function_declarations=[
        tool_get_capital,
        tool_get_exchange_rate,
    ],
)

# Model Initialization with tools
model = GenerativeModel(MODEL_NAME,
                        generation_config={"temperature": 0},
                        tools=[search_tool])


## Function call

In [26]:
def function_call(chat, prompt:str)->str:

  # 1. Find functions to be called corresponding to prompt.
  response = chat.send_message(prompt)

  print(f"0. Original response: {response}")

  function_call = response.candidates[0].content.parts[0].function_call

  print(f"1. Find functions to be called: {function_call}")

  function_handlers = {
      "get_capital": get_capital,
      "get_exchange_rate": get_exchange_rate,
  }

  # 2. Loop to call tool functions.
  if function_call is None:
      return "No function to be called"

  if function_call.name in function_handlers:
      function_name = function_call.name
      chat_response = ""

      # get args lists from function_call.
      args = {key: value for key, value in function_call.args.items()}
      print(f"2-1. function args: {args}")

      if args:

          function_response = function_handlers[function_name](args)
          print(f"2-2. function response: {function_response}")

          # build a part data with function response
          part_data = Part.from_function_response(
                  name=function_name,
                  response={"content": function_response,}
          )
          print(f"2-3. part_data: {part_data}")

          response = chat.send_message(part_data,)
          chat_response = response.candidates[0].content.parts[0].text
          print(f"2-4. Final response with function calling: {chat_response}")

          #return chat_response
      else:
          print("No arguments found for the function.")
          return "Elaborate a question again to call tools"
  else:
      print("Responsed without function calling")
      return response.text

  return chat_response

### Execute function call

In [27]:
# Find a function.
prompt = """ What is the capital city of South Korea?"""
chat = model.start_chat()

response = function_call(chat, prompt )

print(response)

0. Original response: candidates {
  content {
    role: "model"
    parts {
      function_call {
        name: "get_capital"
        args {
          fields {
            key: "country"
            value {
              string_value: "South Korea"
            }
          }
        }
      }
    }
  }
  avg_logprobs: -5.1656642578260892e-07
  finish_reason: STOP
}
model_version: "gemini-1.5-flash-002"
usage_metadata {
  prompt_token_count: 59
  candidates_token_count: 6
  total_token_count: 65
}

1. Find functions to be called: name: "get_capital"
args {
  fields {
    key: "country"
    value {
      string_value: "South Korea"
    }
  }
}

2-1. function args: {'country': 'South Korea'}
2-2. function response: Seoul

2-3. part_data: function_response {
  name: "get_capital"
  response {
    fields {
      key: "content"
      value {
        string_value: "Seoul\n"
      }
    }
  }
}

2-4. Final response with function calling: The capital city of South Korea is Seoul.

The capital c

In [28]:
# Find a function.
prompt = """ What is 2024-12-13 exchange rate between US dollar(USD) and Korean won(KRW), find an answer from tools"""
prompt = "2024-12-20 미국 달러(USD)와 한국 원(KRW)의 환율은 얼마입니까?  Tools 에서 답을 찾으세요"

chat = model.start_chat()

response = function_call(chat, prompt )

print(response)

0. Original response: candidates {
  content {
    role: "model"
    parts {
      function_call {
        name: "get_exchange_rate"
        args {
          fields {
            key: "currency_to"
            value {
              string_value: "KRW"
            }
          }
          fields {
            key: "currency_from"
            value {
              string_value: "USD"
            }
          }
          fields {
            key: "currency_date"
            value {
              string_value: "2024-12-20"
            }
          }
        }
      }
    }
  }
  avg_logprobs: -0.00046734722262179411
  finish_reason: STOP
}
model_version: "gemini-1.5-flash-002"
usage_metadata {
  prompt_token_count: 91
  candidates_token_count: 27
  total_token_count: 118
}

1. Find functions to be called: name: "get_exchange_rate"
args {
  fields {
    key: "currency_to"
    value {
      string_value: "KRW"
    }
  }
  fields {
    key: "currency_from"
    value {
      string_value: "USD"
 